In [2]:
import pandas as pd
import numpy as np
data_set=pd.read_csv('Coffee Shop Sales.csv')

In [3]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

def show_net_sales_by_month_interactive():
    df = data_set.copy()
    # Ensure transaction_date is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['transaction_date']):
        df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
    df = df.dropna(subset=['transaction_date'])
    df['YearMonth'] = df['transaction_date'].dt.to_period('M').astype(str)
    df['MonthOnly'] = df['transaction_date'].dt.strftime('%b')  # e.g., Jan, Feb

    df['net_sales'] = df['transaction_qty'] * df['unit_price']

    # Group by YearMonth and store location
    summary = df.groupby(['YearMonth', 'MonthOnly', 'store_location'], as_index=False)['net_sales'].sum()
    # Pivot for plotting, but also keep the months for the x-axis
    pivot = summary.pivot(index='YearMonth', columns='store_location', values='net_sales')
    months_only = summary.drop_duplicates('YearMonth').set_index('YearMonth')['MonthOnly'].reindex(pivot.index).tolist()

    # Define a custom color palette for stores
    color_palette = [
        '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A',
        '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'
    ]
    store_colors = {}
    for i, store in enumerate(pivot.columns):
        store_colors[store] = color_palette[i % len(color_palette)]

    # Dropdown for interactive selection
    store_options = ['All Stores'] + list(pivot.columns)
    store_dropdown = widgets.Dropdown(
        options=store_options,
        value='All Stores',
        description='Store location:'
    )

    output = widgets.Output()

    def plot(selected_store):
        output.clear_output(wait=True)
        with output:
            fig = go.Figure()
            if selected_store == 'All Stores':
                # Plot a grouped bar for all stores, using the color legend, but remove labels
                for store in pivot.columns:
                    y_data = pivot[store]
                    fig.add_trace(go.Bar(
                        x=months_only,
                        y=y_data,
                        name=store,
                        marker_color=store_colors.get(store, None),
                        text=None,  # Remove bar labels for all stores
                        textposition=None
                    ))
                barmode = 'group'
            else:
                # Plot a single bar trace for the selected store, keeping color consistent with legend
                y_data = pivot[selected_store]
                fig.add_trace(go.Bar(
                    x=months_only,
                    y=y_data,
                    name=selected_store,
                    marker_color=store_colors.get(selected_store, 'lightslategray'),
                    text=[f"${v:,.2f}" if pd.notnull(v) else "" for v in y_data],
                    textposition='outside'
                ))
                barmode = 'group'
            title_str = (
                "Net Sales per Month"
                if selected_store == 'All Stores'
                else f"Net Sales per Month - {selected_store}"
            )
            fig.update_layout(
                barmode=barmode,
                title=title_str,
                xaxis_title="Month",
                yaxis_title="Net Sales ($)",
                legend_title="Store Location",
                # Change the legend style for clarity
                legend=dict(
                    orientation="v",
                    x=1.02,
                    y=1,
                    bgcolor='rgba(255,255,255,0.9)',
                    bordercolor='gray',
                    borderwidth=1,
                    font=dict(size=11)
                ),
                xaxis=dict(
                    tickangle=45,
                    tickmode='array',
                    tickvals=months_only,
                    ticktext=months_only
                ),
                width=900,
                height=500,
                margin=dict(l=60, r=60, t=60, b=60)
            )
            fig.show()

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            plot(change['new'])

    store_dropdown.observe(on_change)
    display(widgets.VBox([store_dropdown, output]))
    plot(store_dropdown.value)

# Run the interactive function to display the bar chart with dropdown and improved color legend
show_net_sales_by_month_interactive()


In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd

import plotly.graph_objs as go

# Prepare initial dropdown options for stores and categories (no product type dropdown)
store_locations = ['All Stores'] + sorted(data_set['store_location'].unique())
product_categories = ['All Categories'] + sorted(data_set['product_category'].unique())

initial_store = 'All Stores'
initial_category = 'All Categories'

store_dropdown = widgets.Dropdown(
    options=store_locations,
    value=initial_store,
    description='Store location:'
)
category_dropdown = widgets.Dropdown(
    options=product_categories,
    value=initial_category,
    description='Product category:'
)

output_chart = widgets.Output()

def get_filtered_data(store_location, product_category):
    df = data_set.copy()
    if store_location != 'All Stores':
        df = df[df['store_location'] == store_location]
    if product_category != 'All Categories':
        df = df[df['product_category'] == product_category]
    return df

def get_monthly_category_summary(store_location, product_category):
    df_filtered = get_filtered_data(store_location, product_category).copy()
    if df_filtered.empty:
        return pd.DataFrame()

    # Parse the transaction_date as datetime
    df_filtered['transaction_date'] = pd.to_datetime(df_filtered['transaction_date'], format='%d/%m/%Y')
    # Extract year-month for grouping
    df_filtered['month'] = df_filtered['transaction_date'].dt.to_period('M').astype(str)
    # Group by product_category and month, sum transaction_qty
    # If viewing "All Categories", show all product categories as separate lines
    if product_category == 'All Categories':
        summary = (
            df_filtered
            .groupby(['product_category', 'month'], as_index=False)['transaction_qty']
            .sum()
            .rename(columns={'transaction_qty': 'Total Ordered Quantity'})
        )
        # only keep the top 7 categories overall
        cat_totals = summary.groupby('product_category')['Total Ordered Quantity'].sum().sort_values(ascending=False).head(7)
        top_cats = cat_totals.index.tolist()
        summary = summary[summary['product_category'].isin(top_cats)]
        group_col = 'product_category'
    else:
        # Single chosen category: sum to a single line
        summary = (
            df_filtered.groupby(['month'], as_index=False)['transaction_qty']
            .sum()
            .rename(columns={'transaction_qty': 'Total Ordered Quantity'})
        )
        summary['product_category'] = product_category
        group_col = 'product_category'

    # Ensure the time axis is sorted (all months included for each group)
    months = sorted(summary['month'].unique())
    all_groups = summary[group_col].unique()
    complete_idx = pd.MultiIndex.from_product([all_groups, months], names=[group_col, 'month'])
    summary = summary.set_index([group_col, 'month']).reindex(complete_idx, fill_value=0).reset_index()
    summary = summary.sort_values([group_col, 'month'])
    return summary

# Define a color palette for categories (for Plotly traces and matplotlib)
category_colors = {
    'Coffee':'#4C6058',
    'Tea':'#7A4419',
    'Drinking Chocolate':'#924C9D',
    'Bakery':'#E3AC4E',
    'Flavours':'#D94C4C',
    'Loose Tea':'#32A69B',
    'Coffee beans':'#86624E',
    'Packaged Chocolate':'#8376B6',
    'Branded':'#E37C96'
}

def _remove_2023_from_title(title):
    # Remove 2023 year from the string (both "2023-" and "2023/" formats, and parenthesis/CSV possible)
    import re
    # Remove whole year '2023' or '2023-' or '2023/' at start or in parentheses
    new_title = re.sub(r'(2023[/-]?)', '', title)
    new_title = re.sub(r'\(*2023\)*', '', new_title)
    # Remove standalone '2023' with/without parentheses/commas
    new_title = re.sub(r'[,\s]*2023[,\s]*', '', new_title)
    new_title = re.sub(r'\(\s*,\s*\)', '', new_title)
    return new_title.strip()

def update_chart(change=None):
    with output_chart:
        clear_output(wait=True)
        summary = get_monthly_category_summary(
            store_dropdown.value,
            category_dropdown.value
        )
        if summary.empty:
            print("No data available for the selected filters.")
            return

        if category_dropdown.value == 'All Categories':
            pivot = summary.pivot(index='month', columns='product_category', values='Total Ordered Quantity').fillna(0)
            # Sort columns/categories by total orders DESCENDING for hover (bar) order!
            cat_order = pivot.sum(axis=0).sort_values(ascending=False).index.tolist()
            pivot = pivot[cat_order]
            columns_to_plot = list(pivot.columns)
            legend_title = "Product Category"
        else:
            # Only the selected category, single line
            pivot = summary.pivot(index='month', columns='product_category', values='Total Ordered Quantity').fillna(0)
            columns_to_plot = [category_dropdown.value]
            legend_title = "Product Category"

        months = list(pivot.index)

        # For axis: Remove '2023-' or leading '2023/' or trailing ' 2023', etc. from month labels (optional)
        def format_month_label(month):
            # If month like '2023-05' or '2023/05', remove the year part
            if isinstance(month, str) and (month.startswith('2023-') or month.startswith('2023/')):
                return month[5:]
            return month

        months_no_year = [format_month_label(m) for m in months]

        # Get month abbreviations for x-axis (to add to bar titles)
        # Parse the index (which should be like '2023-05' or similar)
        import calendar
        def get_month_abbr(m):
            # m expected like '2023-05' or '05' after stripping year
            if isinstance(m, str):
                if '-' in m:
                    part = m.split('-')[-1]
                elif '/' in m:
                    part = m.split('/')[-1]
                else:
                    part = m
                try:
                    m_int = int(part)
                    return calendar.month_abbr[m_int]
                except Exception:
                    return m
            return str(m)

        months_abbr = [get_month_abbr(m) for m in months]

        try:
            fig = go.Figure()
            for i, col in enumerate(columns_to_plot):
                color = category_colors.get(col, f'rgba(60,60,{(i*40)%255},0.97)')
                fig.add_trace(go.Scatter(
                    x=months_abbr,
                    y=pivot[col],
                    mode='lines+markers',
                    name=col,
                    line=dict(width=2, color=color),
                    marker=dict(size=8, color=color),
                    hovertemplate='%{x}<br>%{y:,d}<br>%{fullData.name}<extra></extra>',
                    opacity=0.7
                ))

            # Only show month abbreviations as tick labels
            axis_ticktext = months_abbr

            # Build the title string as before, but remove 2023 from context
            raw_title = f"Monthly Orders per Product Category<br><sup>({store_dropdown.value}, {category_dropdown.value})</sup>"
            filtered_title = _remove_2023_from_title(raw_title)

            fig.update_layout(
                width=850, height=420,
                title=filtered_title,
                xaxis=dict(
                    title='Month',
                    tickmode='array',
                    tickvals=months_abbr,
                    ticktext=axis_ticktext
                ),
                yaxis=dict(title='Total Ordered Quantity', tickformat=',d'),
                legend=dict(title=legend_title, x=1.03, y=1, font=dict(size=10), traceorder="normal"),
                margin=dict(l=60, r=160, t=70, b=50),
                hovermode='x unified',
            )

            try:
                from plotly.graph_objs import FigureWidget
                fw = FigureWidget(fig)
                display(fw)
            except ImportError:
                import plotly.io as pio
                pio.show(fig)
        except Exception as e:
            plt.figure(figsize=(11, 5))
            color_cycle = []
            for col in columns_to_plot:
                color = category_colors.get(col, None)
                color_cycle.append(color)
            # fallback to default color cycle if not enough custom
            for i, col in enumerate(columns_to_plot):
                plt.plot(months_abbr, pivot[col], marker='o', label=col, linewidth=2,
                         color=color_cycle[i] if color_cycle[i] else None)
            # Remove 2023 from the title context
            raw_title = f"Monthly Orders per Product Category\n({store_dropdown.value}, {category_dropdown.value})"
            filtered_title = _remove_2023_from_title(raw_title)
            plt.title(filtered_title)
            # Set new x-labels: only month abbreviations
            plt.xlabel('Month')
            plt.ylabel('Total Ordered Quantity')
            plt.xticks(ticks=range(len(months_abbr)), labels=months_abbr)
            plt.legend(title=legend_title, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
            import matplotlib.ticker as ticker
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
            plt.tight_layout()
            plt.show()
            print(f"Displayed static chart (Plotly FigureWidget not available: {e})")

store_dropdown.observe(update_chart, names='value')
category_dropdown.observe(update_chart, names='value')

display(widgets.VBox([store_dropdown, category_dropdown, output_chart]))
update_chart()


In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Extract hour if not already present
if 'hour' not in data_set.columns:
    data_set['hour'] = data_set['transaction_time'].str.split(':').str[0].astype(int)

# Prepare unique sorted store locations for dropdown
hour_store_locations = ['All Stores'] + sorted(data_set['store_location'].unique())

hour_store_dropdown = widgets.Dropdown(
    options=hour_store_locations,
    value='All Stores',
    description='Store location:'
)

hour_output_chart = widgets.Output()

def plot_hourly_transactions(store_loc):
    if store_loc == 'All Stores':
        df = data_set
    else:
        df = data_set[data_set['store_location'] == store_loc]
    # Group by hour and sum transaction_qty
    hourly_summary = df.groupby('hour', as_index=False)['transaction_qty'].sum()
    # Sort by hour ascending
    hourly_summary = hourly_summary.sort_values('hour')
    if hourly_summary.empty:
        print("No data available for the selected store location.")
        return
    plt.figure(figsize=(10, 4))
    bars = plt.bar(hourly_summary['hour'], hourly_summary['transaction_qty'], color='coral')
    plt.xlabel('Hour of Day')
    plt.ylabel('Total Quantity Ordered')
    plt.title(
        f"Total Transactions by Hour - {store_loc}"
        if store_loc != 'All Stores'
        else "Total Transactions by Hour - All Stores"
    )
    plt.xticks(hourly_summary['hour'])
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    # Add labels on top of bars
    for bar, qty in zip(bars, hourly_summary['transaction_qty']):
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            height,
            str(qty),
            ha='center',
            va='bottom',
            fontsize=9
        )
    plt.tight_layout()
    plt.show()

def update_hour_chart(change=None):
    with hour_output_chart:
        clear_output(wait=True)
        plot_hourly_transactions(hour_store_dropdown.value)

hour_store_dropdown.observe(update_hour_chart, names='value')
display(widgets.VBox([hour_store_dropdown, hour_output_chart]))
update_hour_chart()
